In [47]:
import math
import random

In [48]:
class Tree(object):
    def __init__(self, data, left = None, right = None):
        self.left = left
        self.right = right
        self.data = data
    
    def __str__(self):
        return str(self.data)
    
    def getTerminals(self, util = []):
        terminals = []
        for i in util:
            terminals.append(i)
        
        if self.left != None:
            terminals = self.left.getTerminals(terminals)
        if self.right != None:
            terminals = self.right.getTerminals(terminals)
        if self.left == None and self.right == None:
            terminals.append(self.data)
        
        return terminals
    
    def printAll(self, numSpaces = 0):
        if self.right != None:
            self.right.printAll(numSpaces + 10)
            print("\n")
        
        for i in range(numSpaces):
            print(end = " ")
        print(self)
        
        if self.left != None:
            print("\n")
            self.left.printAll(numSpaces + 10)

In [49]:
class Jet(Tree):
    E_INITIAL = 8000
    E_CRIT = 0.17
    
    def randZ():
        y = random.random()
        z = 2 ** y - 1
        
        return z
    
    def randTheta():
        y = random.random()
        theta = (1 + math.pi / 2) ** y - 1
        
        return theta
    
    def randPhi():
        return math.pi * random.random()
    
    def pInitial():
        e = - Jet.E_INITIAL * math.log(1 - random.random())
        theta = math.pi * random.random()
        phi = 2 * math.pi * random.random()
        p1 = [0, 0, 0, 0]
        p2 = [0, 0, 0, 0]
        
        p1[0] = e
        p1[1] = e * math.sin(theta) * math.cos(phi)
        p1[2] = e * math.sin(theta) * math.sin(phi)
        p1[3] = e * math.cos(theta)
        
        p2[0] = p1[0]
        for i in [1, 2, 3]: p2[i] = - p1[i]
        
        return [p1, p2]
    
    def radiate(p):
        rad = [0, 0, 0, 0]
        z = Jet.randZ()
        theta = Jet.randTheta()
        phi = Jet.randPhi()
        s = math.sqrt(p[1] ** 2 + p[2] ** 2)
        
        rad[0] = z * p[0]
        if s != 0:
            rad[1] = z * ((p[1] * p[3] * math.sin(theta) * math.cos(phi) - p[2] * p[0] * math.sin(theta) * math.sin(phi)) / s + p[1] * math.cos(theta))
            rad[2] = z * ((p[2] * p[3] * math.sin(theta) * math.cos(phi) + p[1] * p[0] * math.sin(theta) * math.sin(phi)) / s + p[2] * math.cos(theta))
        else:
            rad[1] = z * (p[3] * math.sin(theta) * math.cos(phi) + p[1] * math.cos(theta))
            rad[2] = z * (p[0] * math.sin(theta) * math.sin(phi) + p[2] * math.cos(theta)) 
        rad[3] = z * (p[3] * math.cos(theta) - s * math.sin(theta) * math.cos(phi))
        
        return rad
    
    def __init__(self, data):
        super().__init__(data)
        if data[0] > Jet.E_CRIT:
            rad = Jet.radiate(data)
            f = [0, 0, 0, 0]
            
            for i in range(4):
                f[i] = data[i] - rad[i]
            self.left = Jet(rad)
            self.right = Jet(f)
    
    def pT(data):
        return math.sqrt(data[1] ** 2 + data[2] ** 2)
    
    def phi(data):
        x = data[1]
        y = data[2]
        
        if x == 0:
            phi = math.pi * y / (2 * abs(y))
        elif y == 0:
            return (math.pi * ((1 - x / abs(x)) / 2))
        else:
            phi = math.atan(y / x) + math.pi * ((1 - x / abs(x)) / 2) * (y / abs(y))
        
        return phi
    
    def eta(data):
        s = math.sqrt(data[1] ** 2 + data[2] ** 2)
        z = data[3]
        p = math.sqrt(s ** 2 + z ** 2)
        
        return (- math.log((s / p) / (1 + z / p)))
    
    def delta_ij(i, j):
        phi_i = Jet.phi(i)
        phi_j = Jet.phi(j)
        eta_i = Jet.eta(i)
        eta_j = Jet.eta(j)
        
        return math.sqrt((phi_i - phi_j) ** 2 + (eta_i - eta_j) ** 2)
    
    def d_ij(i, j, p, R):
        return (min(Jet.pT(i) ** (2 * p), Jet.pT() ** (2 * p)) * Jet.delta_ij(i, j) / R)
    
    def d_iB(i, p):
        return (Jet.pT(i) ** (2 * p))
    
    def cluster(utilPJ, utilJ = [], p = 1, R = 1.0):
        if utilPJ == []: return utilJ
        
        pj = []
        j = []
        for i in utilPJ:
            pj.append(i)
        for i in utilJ:
            j.append(i)
        
        min_i = [len(pj) - 1, Jet.d_iB(pj[len(pj) - 1], p)]
        for i in range(len(pj) - 1):
            d = Jet.d_iB(pj[i], p)
            if d < min_i[1]: min_i = [i, d]
        
        min_ij = [[1, 0], Jet.d_ij(pj[1], pj[0], p, R)]
        for i in range(len(pj)):
            for j in range(i):
                d = Jet.d_ij(pj[i], pj[j], p, R)
                if d < min_ij[1]: min_ij = [[i, j], d]
        
        if min_i[1] < min_ij[1]:
            j.append(pj[min_i[0]])
            del pj[min_i[0]]
        else:
            cluster = [0, 0, 0, 0]
            for i in range(4): cluster[i] = pj[min_ij[0][0]][i] + pj[min_ij[0][1]][i]
            
            pj.append(cluster)
            del pj[min_ij[0][0]]
            del pj[min_ij[0][1]]
        
        return Jet.cluster(pj, j, p, R)

In [54]:
jets = []
numJets = []

for i in range(1000):
    p = Jet.pInitial()
    print(p[0][0])
    h = []
    h1 = Jet(p[0]).getTerminals()
    h2 = Jet(p[1]).getTerminals()
    
    for i in h1:
        h.append(i)
    for i in h2:
        h.append(i)
    jets.append(cluster(h))
    numJets.append(len(jets[i]))

10402.56361104324


KeyboardInterrupt: 